<a href="https://colab.research.google.com/github/ashutosh3060/friday-burger-mojito/blob/master/data_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of Contents

0. Libraries
1. User-Defined Functions
2. Import Data and Basic Understanding
3. Data Quality Checks   
  3.1Duplicate    
        3.1.1 Complete row duplicate
        3.1.2 Unique Id duplicate
  3.2 Missing value
4. Feature Engineering   
  4.1 Independent Feature Creation   
  4.2 Join all independently created datasets      
5. Save Final Dataset having new features

## 0. Libraries 

In [1]:
# warnings
import warnings
warnings.filterwarnings("ignore")

# Numpy, Pandas
import numpy as np
import pandas as pd

# Display settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 15)

## 1. User-defined Functions

In [63]:
def null_perc_check (df):
    '''
    Calculates missing value count and percentage for all the columns in a dataframe

    Inputs
    -------
    df : dataframe
        The dataframe for which missing value distribution needs to checked

    Output
    -------
    dataframe
        a dataframe showing missing value count and percentage for all the columns
    '''
    missing_value_df = pd.DataFrame(index = df.keys(), data =df.isnull().sum(), columns = ['Missing_Value_Count'])
    missing_value_df['Missing_Value_Percentage'] = np.round(((df.isnull().mean())*100),1)
    sorted_df = missing_value_df.sort_values('Missing_Value_Count',ascending= False)
    return sorted_df

def order_dist_perc_calc(df, unique_id, feature):   
    '''
    Calculates the order % for unique categories from the given independent variable
    
    Inputs
    ----------
    df : dataframe
        The raw dataframe which is used for new feature creation
    unique_id : string
        Unique id name
    feature : string
        Name of the feature for which order % of unique categories need to be calculated

    Output
    -------
    dataframe
        a dataframe showing order percentage of unique categories for each customer
    '''
    feature_grp = df.groupby([unique_id, feature]).agg({feature: 'count'})
    feature_pct = feature_grp.groupby(level=0).apply(lambda x: round(100 * x / float(x.sum()),2))
    feature_pct.rename(columns={feature: f"{feature}_pct"}, inplace=True)
    feature_id = feature_pct.reset_index()
    df_feature_id = pd.DataFrame(feature_id.pivot(index=unique_id, columns=feature, values=f'{feature}_pct'))
    df_feature_id.reset_index()
    df_feature_id = df_feature_id.fillna(0)
    df_feature_id = df_feature_id.apply(pd.to_numeric)
    df_feature_id = df_feature_id.add_prefix(f'{feature}_')
    return df_feature_id

## 2. Import Data and Basic Understanding

In [3]:
# Define the path, filename

order_path = "/content/"
order_file = "machine_learning_challenge_order_data.csv"
label_path = "/content/"
label_file = "machine_learning_challenge_labeled_data.csv"

In [4]:
# import the dataset as a dataframe

df_order = pd.read_csv(order_path+order_file)
df_label = pd.read_csv(label_path+label_file)

In [7]:
# df_order: shape and first few records

print(df_order.shape)
df_order.head(3)

(786600, 13)


,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
0,000097eabfd9,2015-06-20,19,1.0,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356
1,0000e2c6d9be,2016-01-29,20,1.0,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356
2,000133bb597f,2017-02-26,19,1.0,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324


In [8]:
# df_label: shape and first few records

print(df_label.shape)
df_label.head(3)

(245455, 2)


,customer_id,is_returning_customer
0,000097eabfd9,0
1,0000e2c6d9be,0
2,000133bb597f,1


In [9]:
# Number of unique customers in both the datasets

print(f'df_order: {df_order.customer_id.nunique()}')
print(f'df_label: {df_label.customer_id.nunique()}')

df_order: 245455
df_label: 245455


* As mentioned in the problem document, it seems like multiple records are present for a single customer (Historical data)

* In both order and label datasets, #unique customers are same.

* Before merging both the datasets, need to check the intersection of the unique customer_id

In [10]:
# Intersection of unique customers in both the datasets

cust_intersect = pd.merge(df_order["customer_id"], df_label["customer_id"], how='inner', on=['customer_id'])
print(f'Number of common unique customers in both the datasets: {cust_intersect["customer_id"].nunique()}')

Number of common unique customers in both the datasets: 245455


* This confirms, all the unique customer_id in both the datasets are same.

* So, I will try to extract all the features by grouping, aggregating and other thought process. And then will merge the target variable from the label dataset

* For now, I can make the df_order as the primary dataset for independent feature extraction.

In [11]:
# Datatypes overview of the order dataset

df = df_order.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786600 entries, 0 to 786599
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   customer_id          786600 non-null  object 
 1   order_date           786600 non-null  object 
 2   order_hour           786600 non-null  int64  
 3   customer_order_rank  761833 non-null  float64
 4   is_failed            786600 non-null  int64  
 5   voucher_amount       786600 non-null  float64
 6   delivery_fee         786600 non-null  float64
 7   amount_paid          786600 non-null  float64
 8   restaurant_id        786600 non-null  int64  
 9   city_id              786600 non-null  int64  
 10  payment_id           786600 non-null  int64  
 11  platform_id          786600 non-null  int64  
 12  transmission_id      786600 non-null  int64  
dtypes: float64(4), int64(7), object(2)
memory usage: 78.0+ MB


* Only **customer_order_rank**  has missing values.
* order_date is object, datatype casting is needed for operation.

In [21]:
# Datatypes overview of the customer label dataset

df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245455 entries, 0 to 245454
Data columns (total 2 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   customer_id            245455 non-null  object
 1   is_returning_customer  245455 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ MB


* No missing values
* Also the datatype seems okay.

## 3. Data Quality Checks

### 3.1 Duplicate record checks

#### 3.1.1 Complete Row Duplicate

In [12]:
# Check duplicates

check_dup = df.duplicated(subset=None, keep='first')
print(f'Number of Total records:  {df.shape[0]}')
print(f'Number of Unique records: {check_dup.shape[0]}')
print(f'Number of Duplicates: {df.shape[0] - check_dup.shape[0]}')

Number of Total records:  786600
Number of Unique records: 786600
Number of Duplicates: 0


* Both the total and unique records are same, so there are no duplicate present.

#### 3.1.2 customer_id (unique identifier) Duplicate

In [13]:
# Number of records for each customer_id

df["customer_id"].value_counts()

15edce943edd    386
8745a335e9cf    288
d956116d863d    286
0063666607bb    273
ae60dce05485    270
               ... 
ba7d5a6e6638      1
4324cbb32eb8      1
9aafd98d48b6      1
bea68ff2ad49      1
ca16c3a1e1af      1
Name: customer_id, Length: 245455, dtype: int64

* As mentioned in the Data Dictionary, a customer_id can appear for multiple times as multiple orders in different dates are present, so a valid business scenario.

### 3.2 Null check - Missing value percentage

In [16]:
# Missing value count and percentage check

null_perc_check(df)

,Missing_Value_Count,Missing_Value_Percentage
customer_order_rank,24767,3.1
customer_id,0,0.0
order_date,0,0.0
order_hour,0,0.0
is_failed,0,0.0
voucher_amount,0,0.0
delivery_fee,0,0.0
amount_paid,0,0.0
restaurant_id,0,0.0
city_id,0,0.0


* There are no missing values except for customer_order_rank.
* customer_order_rank can be null whenever an order is failed.
* Once this condition is checked, we can confirm the actual missing percentage of customer_order_rank

In [18]:
# Check number of failed orders and the customer_order_rank for the same

print(f'#Failed orders: {df.loc[df["is_failed"]==1].shape[0]}')
print(f'unique customer_rank_order values:  {df.loc[df["is_failed"]==1]["customer_order_rank"].unique()}')

#Failed orders: 24767
unique customer_rank_order values:  [nan]


1. #Failed orders = #Missing values for customer_order_rank
2. For all failed orders, customer_order_rank is null    
* Above 2 statements prove that the Nulls are business case wise correct; hence these are not semantically missing values.

In [19]:
# Check the missing value percentage for label dataset

null_perc_check(df_label)

,Missing_Value_Count,Missing_Value_Percentage
customer_id,0,0.0
is_returning_customer,0,0.0


* There are no missing values for customer label dataset.

## 4. Feature Engineering

### 4.1 Independent Feature Creation

In [22]:
df.columns

Index(['customer_id', 'order_date', 'order_hour', 'customer_order_rank',
       'is_failed', 'voucher_amount', 'delivery_fee', 'amount_paid',
       'restaurant_id', 'city_id', 'payment_id', 'platform_id',
       'transmission_id'],
      dtype='object')

In [30]:
df.tail(5)

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
786595,fffe9d5a8d41,2016-09-30,20,NaN,1,0.0,0.0,10.7262,983498,10346,1779,29463,212
786596,ffff347c3cfa,2016-08-17,21,1.0,0,0.0,0.0,7.5933,52893498,41978,1619,30359,4356
786597,ffff347c3cfa,2016-09-15,21,2.0,0,0.0,0.0,5.9472,164653498,41978,1619,30359,4356
786598,ffff4519b52d,2016-04-02,19,1.0,0,0.0,0.0,21.7710,16363498,80562,1491,29751,4228
786599,ffffccbfc8a4,2015-05-30,20,1.0,0,0.0,0.0,16.4610,150293498,45952,1619,29463,4324


### List of features to be created

**1. How old the customer in business according to order_date**   
i.e. need to extract customer's first order date within this 2 years time span   
**2. From how many days Customer has not ordered**   
i.e. Last order before how many days   
**3. Customer ordered for a period of n days (continuously)**   
i.e. Difference between first_order_date and last_order_date    
**4. Most Common order hour**   
i.e. Most occuring / mode of order_hour   
**5. Number of successful orders**   
i.e. count of customer_order_rank (need to exclude null values)   
**6. Number of failed orders**   
i.e. count of is_failed==1    
**7. Average voucher amount used**      
i.e. Mean of voucher amount    
**8. Average delivery fee paid**    
i.e. Mean of delivery_fee       
**9. Average amount paid**    
i.e. Mean of voucher amount   
**10. Customer has ordered from how many unique number of restaurants**     
i.e. # unique restaurant_id         
**11. Customer has ordered from how many unique number of cities**     
i.e. # unique city_id     
**12. Customer has used how many unique number of payment_id**     
i.e. # unique payment_id     
**13. Customer has ordered from how many unique number of platforms**     
i.e. # unique platform_id         
**14. Customer has used how many unique number of transmission_id**  
i.e. Need to create categorical encoding type features for each unique transmission_id and need to assign the respective percentage of use   
**15. Has customer used any voucher**       
i.e. if voucher_amount==0 then not used else used      
**16. Has customer paid any delivery fee**       
i.e. if delivery_fee==0 then not paid else paid       
**17. Has customer paid any amount ever**    
i.e. if amount_paid==0 then not paid else paid      
**18 What is the order % distribution according to day-time**   
i.e. How much orders are in morning, afternoon, evening etc (Need to assume day-time range according to business)


* Need to think, how to process the failed and successful orders

In [33]:
## Create copy of original dataset for new feature creation
df_ftr_crtn = df.copy()
print(df.shape)
print(df_ftr_crtn.shape)

## Typecasting the date field for operation
print(df_ftr_crtn["order_date"].dtypes)
df_ftr_crtn["order_date"] = pd.to_datetime(df_ftr_crtn["order_date"])
print(df_ftr_crtn["order_date"].dtypes)

(786600, 13)
(786600, 13)
object
datetime64[ns]


1. How old the customer in business according to order_date
2. From how many days Customer has not ordered 
3. Customer ordered for a period of n days (continuously)  

In [41]:
# days before first, last order and period of continuous orders 
#(Including failed order for simplicity. This shows customer first interacted and last interacted irrespective of failed order)

min_max_order_date = pd.DataFrame(df_ftr_crtn.groupby("customer_id")["order_date"].agg(['min', 'max'])).reset_index()
min_max_order_date["first_order_before"] = (pd.to_datetime('2017-03-01') - min_max_order_date["min"]).dt.days
min_max_order_date["last_order_before"] = (pd.to_datetime('2017-03-01') - min_max_order_date["max"]).dt.days
min_max_order_date["cntns_order_period"] = np.where((min_max_order_date["first_order_before"] - min_max_order_date["last_order_before"])==0, 1, min_max_order_date["first_order_before"] - min_max_order_date["last_order_before"])
min_max_order_date.rename(columns={"min":"first_order_date", "max":"last_order_date"}, inplace=True)
min_max_order_date.tail()

,customer_id,first_order_date,last_order_date,first_order_before,last_order_before,cntns_order_period
245450,fffd696eaedd,2015-09-14,2015-09-14,534,534,1
245451,fffe9d5a8d41,2016-07-31,2016-09-30,213,152,61
245452,ffff347c3cfa,2016-08-17,2016-09-15,196,167,29
245453,ffff4519b52d,2016-04-02,2016-04-02,333,333,1
245454,ffffccbfc8a4,2015-05-30,2015-05-30,641,641,1


6. Number of failed orders

In [42]:
# Number of Failed orders
num_fail_order = pd.DataFrame(df_ftr_crtn.groupby("customer_id")["is_failed"].agg(['sum'])).reset_index()
num_fail_order.rename(columns={"sum":"num_fail_order"}, inplace=True)
num_fail_order.tail(5)

,customer_id,num_fail_order
245450,fffd696eaedd,0
245451,fffe9d5a8d41,2
245452,ffff347c3cfa,0
245453,ffff4519b52d,0
245454,ffffccbfc8a4,0


* Can we exclude the customers having failed orders can be excluded for other new features creation ?

In [44]:
# Check record count excluding all thge failed cases

print(df_ftr_crtn.shape)
df_ftr_crtn = df_ftr_crtn.loc[df_ftr_crtn["is_failed"]==0]
print(df_ftr_crtn.shape)

(761833, 13)
(761833, 13)


In [46]:
# Create another dataset by keeping only single customer_id

df_new_ftr_1 = df_ftr_crtn.drop_duplicates(subset="customer_id", keep="first")
print(df_new_ftr_1.shape)

(244937, 13)


In [47]:
# Total unique customers and customers having only failed records

print('Total unique number of customers:')
print(df.customer_id.nunique())
print('--'*20)
print('Records having only successful orders of customers:')
print(df_new_ftr_1.customer_id.nunique())
print('--'*20)
print('Customers having only failed orders:')
print(df.customer_id.nunique() - df_new_ftr_1.shape[0])

Total unique number of customers:
245455
----------------------------------------
Records having only successful orders of customers:
244937
----------------------------------------
Customers having only failed orders:
518


* After excluding customers who has only failed orders, but no successful order: we get 244937 unique customers.
* Let's check if there are any returning customers in those 518 records.

In [48]:
# Create list of unique customers from both the datasets
list_df_customer_id = list(df.customer_id.unique())
list_df_ftr_crtn_customer_id = list(df_ftr_crtn.customer_id.unique())

# Create set for 518 customers which are having only failed records
fail_customers = set(list_df_customer_id) - set(list_df_ftr_crtn_customer_id)
# Check what is their returning status in label dataset
df_label.loc[df_label["customer_id"].isin(list(fail_customers))]["is_returning_customer"].value_counts()

1    488
0     30
Name: is_returning_customer, dtype: int64

* Let's check for a sample record

In [49]:
df_fail_customer = pd.DataFrame(fail_customers)
df_fail_customer.head()

,0
0,2f927c7cd42e
1,20c4908fd6cb
2,c8d828742879
3,2182883b7600
4,4e03febae7de


In [54]:
print(df_label.loc[df_label["customer_id"]=="2f927c7cd42e"]["is_returning_customer"])
print('--'*25)
print('--'*25)
df[df["customer_id"]=="2f927c7cd42e"]

45896    1
Name: is_returning_customer, dtype: int64
--------------------------------------------------
--------------------------------------------------


,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
148570,2f927c7cd42e,2016-05-20,21,NaN,1,0.0,0.986,9.6111,187213498,39335,1619,30359,4228


* We can see customer_id "2f927c7cd42e" has **1 failed order in May 2016, but still returning in 2017**.

* This is pretty strange that customers having only failed orders are returning in next 6 months.

* In future, want to analyze the cohort:
         1.what are their first order date, last order date
         2.Is the data correct
         3.Want to understand how customer_id is generated ?

There can be 2 approaches:
1. Exclude the failed ones and build model with successful ones to avoid feeding any misinformation and also bias.
2. Include the failed orders and assume the amount, voucher and other features normal as successful ones.

For 2nd approach it needs proper analysis, an extra pipeline to process/ transform data and also cross-check the accuracy or impact of this on the model.
* For now, keeping an eye on time-constraint I will exclude these 518 records.

4. Common order hour
5. Number of successful orders
7. Average voucher amount used
8. Average delivery fee paid
9. Average amount paid
10. Customer has ordered from how many unique number of restaurants
11. Customer has ordered from how many unique number of cities
12. Customer has used how many unique number of payment_id
13. Customer has ordered from how many unique number of platforms
14. Customer has used how many unique number of transmission_id

In [55]:
df_grp_agg = df_ftr_crtn.groupby('customer_id').agg({'order_hour' : lambda x:x.value_counts().index[0], 
                         'customer_order_rank' : 'count', 
                         'voucher_amount' : 'mean',
                         'delivery_fee' : 'mean',
                         'amount_paid' : 'mean',
                         'restaurant_id' : lambda x:x.nunique(),
                         'city_id' : lambda x:x.nunique(),
                         'payment_id' : lambda x:x.nunique(),
                         'platform_id' : lambda x:x.nunique(),
                         'transmission_id' : lambda x:x.nunique()})
df_grp_agg = df_grp_agg.reset_index()
df_grp_agg.rename(columns={"order_hour": "comm_orderhour", "customer_order_rank": "num_success_order",
                           "voucher_amount": "avg_voucher_amnt", "delivery_fee": "avg_deliveryfee",
                           "amount_paid": "avg_amount", "restaurant_id": "num_restaurant",
                           "city_id" : "num_city", "payment_id" : "num_paymentid",
                           "platform_id" : "num_platform", "transmission_id" : "num_transmission"}, inplace=True)
df_grp_agg.tail()

,customer_id,comm_orderhour,num_success_order,avg_voucher_amnt,avg_deliveryfee,avg_amount,num_restaurant,num_city,num_paymentid,num_platform,num_transmission
244932,fffd696eaedd,12,1,0.0,1.4297,24.13395,1,1,1,1,1
244933,fffe9d5a8d41,20,1,0.0,0.0000,10.72620,1,1,1,1,1
244934,ffff347c3cfa,21,2,0.0,0.0000,6.77025,2,1,1,1,1
244935,ffff4519b52d,19,1,0.0,0.0000,21.77100,1,1,1,1,1
244936,ffffccbfc8a4,20,1,0.0,0.0000,16.46100,1,1,1,1,1


In [56]:
# Cross check aggregation for 1 customer

df_ftr_crtn.loc[df_ftr_crtn["customer_id"]=="ffff347c3cfa"]

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
786596,ffff347c3cfa,2016-08-17,21,1.0,0,0.0,0.0,7.5933,52893498,41978,1619,30359,4356
786597,ffff347c3cfa,2016-09-15,21,2.0,0,0.0,0.0,5.9472,164653498,41978,1619,30359,4356


* Aggregation looks good

15. Has customer used any voucher
16. Has customer paid any delivery fee
17. Has customer paid any amount ever

In [57]:
# % Voucher used vs not used
df_ftr_crtn["is_voucher"] = np.where((df_ftr_crtn["voucher_amount"]==0), "0", df_ftr_crtn["voucher_amount"])
df_ftr_crtn["is_voucher"] = np.where((df_ftr_crtn["voucher_amount"]!=0), "1", df_ftr_crtn["is_voucher"])

# Delivery fee charged or not ?
df_ftr_crtn["is_deliveryfee"] = np.where((df_ftr_crtn["delivery_fee"]==0), "0", df_ftr_crtn["delivery_fee"])
df_ftr_crtn["is_deliveryfee"] = np.where((df_ftr_crtn["delivery_fee"]!=0), "1", df_ftr_crtn["is_deliveryfee"])

# Customers who have never paid
df_ftr_crtn["is_amount"] = np.where((df_ftr_crtn["amount_paid"]==0), "0", df_ftr_crtn["amount_paid"])
df_ftr_crtn["is_amount"] = np.where((df_ftr_crtn["amount_paid"]!=0), "1", df_ftr_crtn["is_amount"])

18. What is the order % distribution according to day-time

In [58]:
# Create new column for order daytime and define various day-time values
df_ftr_crtn["order_daytime"] = np.where((df_ftr_crtn["order_hour"]>=0)&(df_ftr_crtn["order_hour"]<3), "order_late_night", df_ftr_crtn["order_hour"])
df_ftr_crtn["order_daytime"] = np.where((df_ftr_crtn["order_hour"]>=3)&(df_ftr_crtn["order_hour"]<6), "order_early_morning", df_ftr_crtn["order_daytime"])
df_ftr_crtn["order_daytime"] = np.where((df_ftr_crtn["order_hour"]>=6)&(df_ftr_crtn["order_hour"]<=12), "order_morning", df_ftr_crtn["order_daytime"])
df_ftr_crtn["order_daytime"] = np.where((df_ftr_crtn["order_hour"]>12)&(df_ftr_crtn["order_hour"]<17), "order_afternoon", df_ftr_crtn["order_daytime"])
df_ftr_crtn["order_daytime"] = np.where((df_ftr_crtn["order_hour"]>=17)&(df_ftr_crtn["order_hour"]<21), "order_evening", df_ftr_crtn["order_daytime"])
df_ftr_crtn["order_daytime"] = np.where((df_ftr_crtn["order_hour"]>=21)&(df_ftr_crtn["order_hour"]<=23), "order_night", df_ftr_crtn["order_daytime"])

# % distribution of orders in different day-time
order_daytime_grp = df_ftr_crtn.groupby(["customer_id", "order_daytime"]).agg({"order_daytime": "count"})
order_daytime_pct = order_daytime_grp.groupby(level=0).apply(lambda x:
                                                 round(100 * x / float(x.sum()),2))
order_daytime_pct.rename(columns={"order_daytime": "order_daytime_pct"}, inplace=True)
order_daytime = order_daytime_pct.reset_index()
df_order_daytime = pd.DataFrame(order_daytime.pivot(index="customer_id", columns="order_daytime", values="order_daytime_pct"))
df_order_daytime.reset_index()
df_order_daytime = df_order_daytime.fillna(0)
df_order_daytime = df_order_daytime.apply(pd.to_numeric)
df_order_daytime.head()

order_daytime,order_afternoon,order_early_morning,order_evening,order_late_night,order_morning,order_night
customer_id,,,,,,
000097eabfd9,0.0,0.0,100.0,0.0,0.0,0.0
0000e2c6d9be,0.0,0.0,100.0,0.0,0.0,0.0
000133bb597f,0.0,0.0,100.0,0.0,0.0,0.0
00018269939b,0.0,0.0,100.0,0.0,0.0,0.0
0001a00468a6,0.0,0.0,100.0,0.0,0.0,0.0


In [60]:
# Sample record where orders are present in the early morning

df_order_daytime.loc[df_order_daytime["order_early_morning"]!=0].head()

order_daytime,order_afternoon,order_early_morning,order_evening,order_late_night,order_morning,order_night
customer_id,,,,,,
00066870ca92,40.74,3.70,25.93,3.70,0.00,25.93
005cdc3317ed,0.00,20.00,60.00,0.00,20.00,0.00
0078a56bd418,33.33,16.67,0.00,0.00,33.33,16.67
00efda6e39da,0.00,100.00,0.00,0.00,0.00,0.00
01476b72cfa9,10.42,12.50,20.83,4.17,39.58,12.50


In [62]:
# Cross check the order day-time % calculation with 1 sample customer_id

df_ftr_crtn.loc[df_ftr_crtn["customer_id"]=="005cdc3317ed"]

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id,is_voucher,is_deliveryfee,is_amount,order_daytime
1258,005cdc3317ed,2016-04-17,19,1.0,0,0.0,0.000,10.46601,284543498,90633,1619,30231,4228,0,0,1,order_evening
1259,005cdc3317ed,2016-05-08,4,2.0,0,0.0,0.000,6.63750,136523498,90633,1619,30231,4356,0,0,1,order_early...
1260,005cdc3317ed,2016-11-10,19,3.0,0,0.0,0.000,18.05400,195753498,90633,1619,30231,4356,0,0,1,order_evening
1261,005cdc3317ed,2016-11-11,20,4.0,0,0.0,0.493,16.67340,32463498,90633,1619,30231,4324,0,1,1,order_evening
1262,005cdc3317ed,2017-01-21,12,5.0,0,0.0,0.000,23.47020,93173498,90633,1619,29463,4324,0,0,1,order_morning


* Calculation seems okay

* Now order percentage distribution can be calculated based on restaurants, cities, payment method, platform, transmission methods
* Need to check the number of unique values for each of these features

In [68]:
# Number of unique restaurants, cities, payment_id, platform_id, transmission_id used in order

print(df.restaurant_id.nunique(), df.city_id.nunique(), df.payment_id.nunique(), df.platform_id.nunique(), df.transmission_id.nunique())

13569 3749 5 14 10


* **Restaurants** and **Cities** are having so many unique categories, so these 2 can't be used for further % calculation
* Meanwhile **payment_id**, **platform_id** and **transmission_id** are having few unique categories, so these 3 can be used to calculate order % distribution.

19. Distribution of percentage of order according to unique payment_id
20. Distribution of percentage of order according to unique platform_id
21. Distribution of percentage of order according to unique transmission_id

In [64]:
# Create 3 datframes for order % distribution among unique categories for payment_id, platform_id and transmission_id

df_payment_id = order_dist_perc_calc(df_ftr_crtn, "customer_id", "payment_id")
df_platform_id = order_dist_perc_calc(df_ftr_crtn, "customer_id", "platform_id")
df_transmission_id = order_dist_perc_calc(df_ftr_crtn, "customer_id", "transmission_id")

### 4.2 Join all independently created datasets

#### Below dataframes need to be joined for final data preparation
1. df_ftr_crtn
2. min_max_order_date
3. num_fail_order
4. df_grp_agg
5. df_order_daytime
6. df_payment_id
7. df_platform_id
8. df_transmission_id
9. df_label (target table)

In [71]:
df_ftr_crtn.columns

Index(['customer_id', 'order_date', 'order_hour', 'customer_order_rank',
       'is_failed', 'voucher_amount', 'delivery_fee', 'amount_paid',
       'restaurant_id', 'city_id', 'payment_id', 'platform_id',
       'transmission_id', 'is_voucher', 'is_deliveryfee', 'is_amount',
       'order_daytime'],
      dtype='object')

* Define base dataframe for all the joins

In [72]:
# Create the base dataframe for new features from df_ftr_crtn

df_new_ftr = df_ftr_crtn.drop_duplicates(subset="customer_id", keep="first")[["customer_id", "is_voucher", 
                                                                              "is_deliveryfee", "is_amount"]]
print(df_new_ftr.shape)
df_new_ftr.tail(2)

(244937, 4)


,customer_id,is_voucher,is_deliveryfee,is_amount
786598,ffff4519b52d,0,0,1
786599,ffffccbfc8a4,0,0,1


In [73]:
# Left join all the tables with base table (df_new_ftr)
%time

df_new_ftr =  df_new_ftr.merge(min_max_order_date, on='customer_id', how='left').merge(num_fail_order, on='customer_id', how='left')
df_new_ftr =  df_new_ftr.merge(df_grp_agg, on='customer_id', how='left').merge(df_order_daytime, on='customer_id', how='left')
df_new_ftr =  df_new_ftr.merge(df_payment_id, on='customer_id', how='left').merge(df_platform_id, on='customer_id', how='left').merge(df_transmission_id, on='customer_id', how='left')
df_new_ftr =  df_new_ftr.merge(df_label, on='customer_id', how='left')
print(df_new_ftr.shape)
df_new_ftr.tail(2)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
(244937, 55)


,customer_id,is_voucher,is_deliveryfee,is_amount,first_order_date,last_order_date,first_order_before,last_order_before,cntns_order_period,num_fail_order,comm_orderhour,num_success_order,avg_voucher_amnt,avg_deliveryfee,avg_amount,num_restaurant,num_city,num_paymentid,num_platform,num_transmission,order_afternoon,order_early_morning,order_evening,order_late_night,order_morning,order_night,payment_id_1491,payment_id_1523,payment_id_1619,payment_id_1779,payment_id_1811,platform_id_525,platform_id_22167,platform_id_22263,platform_id_22295,platform_id_29463,platform_id_29495,platform_id_29751,platform_id_29815,platform_id_30135,platform_id_30199,platform_id_30231,platform_id_30359,platform_id_30391,platform_id_30423,transmission_id_212,transmission_id_1988,transmission_id_4196,transmission_id_4228,transmission_id_4260,transmission_id_4324,transmission_id_4356,transmission_id_4996,transmission_id_21124,is_returning_customer
244935,ffff4519b52d,0,0,1,2016-04-02,2016-04-02,333,333,1,0,19,1,0.0,0.0,21.771,1,1,1,1,1,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0
244936,ffffccbfc8a4,0,0,1,2015-05-30,2015-05-30,641,641,1,0,20,1,0.0,0.0,16.461,1,1,1,1,1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0


In [74]:
df_new_ftr.columns

Index(['customer_id', 'is_voucher', 'is_deliveryfee', 'is_amount',
       'first_order_date', 'last_order_date', 'first_order_before',
       'last_order_before', 'cntns_order_period', 'num_fail_order',
       'comm_orderhour', 'num_success_order', 'avg_voucher_amnt',
       'avg_deliveryfee', 'avg_amount', 'num_restaurant', 'num_city',
       'num_paymentid', 'num_platform', 'num_transmission', 'order_afternoon',
       'order_early_morning', 'order_evening', 'order_late_night',
       'order_morning', 'order_night', 'payment_id_1491', 'payment_id_1523',
       'payment_id_1619', 'payment_id_1779', 'payment_id_1811',
       'platform_id_525', 'platform_id_22167', 'platform_id_22263',
       'platform_id_22295', 'platform_id_29463', 'platform_id_29495',
       'platform_id_29751', 'platform_id_29815', 'platform_id_30135',
       'platform_id_30199', 'platform_id_30231', 'platform_id_30359',
       'platform_id_30391', 'platform_id_30423', 'transmission_id_212',
       'transmission_id_

In [78]:
# Select the order of feature for better readability and ease of analysis

df_new_ftr = df_new_ftr[['customer_id', 'is_returning_customer', 'first_order_before', 'last_order_before', 'cntns_order_period', 
                         'comm_orderhour', 'num_success_order', 'num_fail_order', 
                         'is_voucher', 'avg_voucher_amnt', 'is_deliveryfee', 'avg_deliveryfee', 'is_amount', 'avg_amount', 
                         'num_restaurant', 'num_city', 'num_paymentid', 'num_platform', 'num_transmission', 
                         'order_afternoon', 'order_early_morning', 'order_evening', 'order_late_night', 'order_morning', 'order_night',
                         'payment_id_1491', 'payment_id_1523',
       'payment_id_1619', 'payment_id_1779', 'payment_id_1811',
       'platform_id_525', 'platform_id_22167', 'platform_id_22263',
       'platform_id_22295', 'platform_id_29463', 'platform_id_29495',
       'platform_id_29751', 'platform_id_29815', 'platform_id_30135',
       'platform_id_30199', 'platform_id_30231', 'platform_id_30359',
       'platform_id_30391', 'platform_id_30423', 'transmission_id_212',
       'transmission_id_1988', 'transmission_id_4196', 'transmission_id_4228',
       'transmission_id_4260', 'transmission_id_4324', 'transmission_id_4356',
       'transmission_id_4996', 'transmission_id_21124']].copy()
# df_new_ftr.index = np.arange(1, len(df_new_ftr)+1)
print(df_new_ftr.shape)
df_new_ftr.tail(2)

(244937, 53)


,customer_id,is_returning_customer,first_order_before,last_order_before,cntns_order_period,comm_orderhour,num_success_order,num_fail_order,is_voucher,avg_voucher_amnt,is_deliveryfee,avg_deliveryfee,is_amount,avg_amount,num_restaurant,num_city,num_paymentid,num_platform,num_transmission,order_afternoon,order_early_morning,order_evening,order_late_night,order_morning,order_night,payment_id_1491,payment_id_1523,payment_id_1619,payment_id_1779,payment_id_1811,platform_id_525,platform_id_22167,platform_id_22263,platform_id_22295,platform_id_29463,platform_id_29495,platform_id_29751,platform_id_29815,platform_id_30135,platform_id_30199,platform_id_30231,platform_id_30359,platform_id_30391,platform_id_30423,transmission_id_212,transmission_id_1988,transmission_id_4196,transmission_id_4228,transmission_id_4260,transmission_id_4324,transmission_id_4356,transmission_id_4996,transmission_id_21124
244935,ffff4519b52d,0,333,333,1,19,1,0,0,0.0,0,0.0,1,21.771,1,1,1,1,1,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
244936,ffffccbfc8a4,0,641,641,1,20,1,0,0,0.0,0,0.0,1,16.461,1,1,1,1,1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0


## 5. Save final dataset having new features

In [76]:
# Export the dataframe to a csv
df_new_ftr.to_csv('customer_order_label.csv', index=False)

In [77]:
# Cross-check the data in csv file

df_customer_order_label = pd.read_csv('customer_order_label.csv')
print(df_customer_order_label.shape)
print('--'*20)
print('--'*20)
print(df_customer_order_label.columns)
print('--'*20)
print('--'*20)
df_customer_order_label.head(3)

(244937, 53)
----------------------------------------
----------------------------------------
Index(['customer_id', 'is_returning_customer', 'first_order_before',
       'last_order_before', 'cntns_order_period', 'comm_orderhour',
       'num_success_order', 'num_fail_order', 'is_voucher', 'avg_voucher_amnt',
       'is_deliveryfee', 'avg_deliveryfee', 'is_amount', 'avg_amount',
       'num_restaurant', 'num_city', 'num_paymentid', 'num_platform',
       'num_transmission', 'order_afternoon', 'order_early_morning',
       'order_evening', 'order_late_night', 'order_morning', 'order_night',
       'payment_id_1491', 'payment_id_1523', 'payment_id_1619',
       'payment_id_1779', 'payment_id_1811', 'platform_id_525',
       'platform_id_22167', 'platform_id_22263', 'platform_id_22295',
       'platform_id_29463', 'platform_id_29495', 'platform_id_29751',
       'platform_id_29815', 'platform_id_30135', 'platform_id_30199',
       'platform_id_30231', 'platform_id_30359', 'platform_id_30

,customer_id,is_returning_customer,first_order_before,last_order_before,cntns_order_period,comm_orderhour,num_success_order,num_fail_order,is_voucher,avg_voucher_amnt,is_deliveryfee,avg_deliveryfee,is_amount,avg_amount,num_restaurant,num_city,num_paymentid,num_platform,num_transmission,order_afternoon,order_early_morning,order_evening,order_late_night,order_morning,order_night,payment_id_1491,payment_id_1523,payment_id_1619,payment_id_1779,payment_id_1811,platform_id_525,platform_id_22167,platform_id_22263,platform_id_22295,platform_id_29463,platform_id_29495,platform_id_29751,platform_id_29815,platform_id_30135,platform_id_30199,platform_id_30231,platform_id_30359,platform_id_30391,platform_id_30423,transmission_id_212,transmission_id_1988,transmission_id_4196,transmission_id_4228,transmission_id_4260,transmission_id_4324,transmission_id_4356,transmission_id_4996,transmission_id_21124
0,000097eabfd9,0,620,620,1,19,1,0,0,0.0,0,0.000,1,11.46960,1,1,1,1,1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
1,0000e2c6d9be,0,397,397,1,20,1,0,0,0.0,0,0.000,1,9.55800,1,1,1,1,1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
2,000133bb597f,1,3,3,1,19,1,0,0,0.0,1,0.493,1,5.93658,1,1,1,1,1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0


* Looks good